#  (New York City) Analyse des trajets en Taxi

### Ressources :

* lien des données : https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

* Description des variables des données sur les trajets en taxi jaune : https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf

* Comment importer un fichier PARQUET de données : https://www.nyc.gov/assets/tlc/downloads/pdf/working_parquet_format.pdf

* Apache Arrow : https://arrow.apache.org/docs/python/parquet.html



#### I/ Automatisation du téléchargement des données

In [9]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq

In [10]:
df = pq.read_table("yellow_tripdata_2024-09.parquet")
type(df)

pyarrow.lib.Table

***"pyarrow.lib.Table" fait reference aux tables SQL***

In [11]:
#conversion en dataframe pandas
df = df.to_pandas()
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2024-09-01 00:05:51,2024-09-01 00:45:03,1.0,9.80,1.0,N,138,48,1,47.8,10.25,0.5,13.30,6.94,1.0,79.79,2.5,1.75
1,1,2024-09-01 00:59:35,2024-09-01 01:03:43,1.0,0.50,1.0,N,140,141,1,5.1,3.50,0.5,3.00,0.00,1.0,13.10,2.5,0.00
2,2,2024-09-01 00:25:00,2024-09-01 00:34:37,2.0,2.29,1.0,N,238,152,2,13.5,1.00,0.5,0.00,0.00,1.0,16.00,0.0,0.00
3,2,2024-09-01 00:31:00,2024-09-01 00:46:52,1.0,5.20,1.0,N,93,130,1,24.7,1.00,0.5,4.55,0.00,1.0,31.75,0.0,0.00
4,2,2024-09-01 00:11:57,2024-09-01 00:30:41,2.0,2.26,1.0,N,79,231,1,17.0,1.00,0.5,4.40,0.00,1.0,26.40,2.5,0.00


In [12]:
df.shape

(3633030, 19)

#### Automatisons à présent le processus de collecte de paquet de données

In [31]:
import requests
import os

In [32]:
#Créons le dossier qui contiendra les paquet de données collectés
os.makedirs('yellow_tripdata_taxis', exist_ok=True)

* Exemple de lien du paquet de données pour Janvier 2024: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet

In [20]:
#Script d'automatisation de la collete des données disponible à date depuis le site web.
for Year in range(2024, 2009, -1):
    for Month in range(1,13):
        # url de téléchargement en fonction de l'année et du mois
        file_url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{Year}-{Month:02d}.parquet"
        file_name = f"yellow_tripdata/yellow_tripdata_{Year}-{Month:02d}.parquet"
        
        # téléchargement du fichier
        print(f"Téléchargement du fichier {file_name}...")
        response = requests.get(file_url, stream=True)
        with open(file_name, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
                else:
                    print(f"Impossible de télécharger {file_name}. Code de statut HTTP: {response.status_code}")

print("Téléchargement terminé!")

Téléchargement du fichier yellow_tripdata/yellow_tripdata_2024-01.parquet...
Téléchargement du fichier yellow_tripdata/yellow_tripdata_2024-02.parquet...
Téléchargement du fichier yellow_tripdata/yellow_tripdata_2024-03.parquet...
Téléchargement du fichier yellow_tripdata/yellow_tripdata_2024-04.parquet...
Téléchargement du fichier yellow_tripdata/yellow_tripdata_2024-05.parquet...
Téléchargement du fichier yellow_tripdata/yellow_tripdata_2024-06.parquet...
Téléchargement du fichier yellow_tripdata/yellow_tripdata_2024-07.parquet...
Téléchargement du fichier yellow_tripdata/yellow_tripdata_2024-08.parquet...
Téléchargement du fichier yellow_tripdata/yellow_tripdata_2024-09.parquet...
Téléchargement du fichier yellow_tripdata/yellow_tripdata_2024-10.parquet...
Téléchargement du fichier yellow_tripdata/yellow_tripdata_2024-11.parquet...
Téléchargement du fichier yellow_tripdata/yellow_tripdata_2024-12.parquet...
Téléchargement du fichier yellow_tripdata/yellow_tripdata_2023-01.parquet...

KeyboardInterrupt: 

**Comptons le nombre total de fichiers téléchargés**

In [23]:
from os import listdir
from os.path import isfile, join

mypatch = "./yellow_tripdata"
files = [f for f in listdir(mypatch) if isfile(join(mypatch, f))]
len(files)

161

#### Mise à jour du script de collecte de données 

* Les données sur les trajets seront publiées chaque mois, mais avec un retard de deux mois par rapport à la période couverte. Par exemple, les données pour Octobre 2024 ne seront publiées qu'en Décembre 2024.

* Pour prendre en compte le fait que les données seront publiées mensuellement et pour maintenir la flexibilité du script, nous pouvons utiliser la date actuelle pour déterminer la fourchette des années à parcourir.

***Voici la version mise à jour du script :***

In [30]:
from datetime import datetime, timedelta


In [ ]:
# Fonction pour obtenir la date de la dernière publication dépendamment du jour actuel
def previous_date_post():
    today = datetime.today()
    two_months_ago = today - timedelta(days=60)
    return two_months_ago.strftime("%Y-%m")


# Année courante
current_year = datetime.now().year
current_month = datetime.now().month

# Previous month
previous_month = previous_date_post()

# Mise à jour du script pour prendre en compte les données à venir
for year in range(2024, current_year + 1):
    for month in range(1, 13):
        # Ignorer les mois antérieurs à la date actuelle moins 2 mois
        if year == current_year and month > current_month:
            continue
        if f"{year}-{month:02d}" < previous_month:
            continue

       # url de téléchargement en fonction de l'année et du mois
        file_url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{Year}-{Month:02d}.parquet"
        file_name = f"yellow_tripdata/yellow_tripdata_{Year}-{Month:02d}.parquet"
        
        # téléchargement du fichier
        print(f"Téléchargement du fichier {file_name}...")
        response = requests.get(file_url, stream=True)
        with open(file_name, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
                else:
                    print(f"Impossible de télécharger {file_name}. Code de statut HTTP: {response.status_code}")

print("Téléchargement terminé!")

Téléchargement du fichier yellow_tripdata_2/yellow_tripdata_2012-06.parquet...
Téléchargement du fichier yellow_tripdata_2/yellow_tripdata_2012-06.parquet...
Téléchargement du fichier yellow_tripdata_2/yellow_tripdata_2012-06.parquet...
Téléchargement terminé!
